# M5のおさらい

- M5ではtreeの形成は普通のdecision treeと同じように行う（？）
    - これは作りたいモデルとはたぶん異なる。
    

# M5PP

- ふたつに分割するアルゴリズムを作る
    - これをrecursiveに適用すればcascading treeになるはず
- 分割にあたっては線形モデルのエラーを最小化するようにする
- ただし、ifelse型のスイッチは最適化困難なので、sigmoidかtanhを用いる。
- 以下のようなmodelを小さい$a$でまずtrainし、徐々に$a$を大きくしていくのがいいだろうか？
　　$$\left[\tanh\left(a(x+b)\right) -1\right]\times\text{回帰式}$$
  
- regime switch用のファクターごとにモデルを作って比較するのが安直ではあるが、もっとスマートな方法はあるか
    - ものすごくきついL1正則項を入れればいい気がする

In [1]:
import tensorflow as tf
import numpy as np

C:\Users\t\Anaconda3\envs\python35\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
Nsample = 1000

In [3]:
Nfactor = 10

In [4]:
trX = np.random.randn(Nsample , Nfactor)

In [5]:
trX.shape

(1000, 10)

In [6]:
Nregime  = 2

In [7]:
A = np.random.randn(Nfactor , Nregime)

In [8]:
trY = 2 * trX + np.random.randn(*trX.shape) * 0.33 # create a y value which is approximately linear but with some random noise

In [9]:
trZ = np.dot(trX , A)

In [10]:
trZ.shape

(1000, 2)

In [11]:
trZ[1].shape

(2,)

In [12]:
Nregime_factor = 1

In [13]:
trX2 = np.random.randn(Nsample, Nregime_factor)

In [14]:
A2 = np.random.randn(Nregime_factor , 1)

In [15]:
trZ2 = np.dot(trX2 , A2)

In [16]:
trX2.shape

(1000, 1)

M5やM5Pに倣い、regime switcherとしては、候補とするファクターひとつのみが含まれる不等式をカスケード式に用いることを想定

従って


In [17]:
trY = [z[0] + z[1] if  trZ2[i , 0] > 0 else z[0] - z[1] for i , z in enumerate(trZ)]

In [23]:
X = tf.placeholder("float", [None, Nfactor])
X2 = tf.placeholder("float", [None, Nregime_factor])

In [57]:
Y = tf.placeholder("float")

## pattern 1

In [ ]:
def model(X, X2 , w, w2): # this network is the same as the previous one except with an extra hidden layer + dropout
    z = tf.matmul(X , w)
    z2 = tf.matmul(X2 , w2)
    h = tf.nn.tanh(z2)
    my_one = tf.constant(dtype="float" , value = 1 )
    weight = tf.subtract(my_one , h) 
    #weight2 = tf.subtract()
    #r = tf.matmul(tf.subtract(my_one , h) , z)
    elementwise_mul = tf.multiply(weight , z)


    return elementwise_mul

In [ ]:
w = tf.Variable(tf.random_normal(shape=[Nfactor , Nregime] , stddev=0.1))

In [ ]:
py = model(X , X2 , w , w2)

## pattern2

In [36]:
def model(X, X2 , w1a , w1b, w2): # this network is the same as the previous one except with an extra hidden layer + dropout
    z1a = tf.matmul(X , w1a)
    z1b = tf.matmul(X , w1b)
    z2 = tf.matmul(X2 , w2)
    h = tf.nn.tanh(z2)
    my_one = tf.constant(dtype="float" , value = 1 )
    weight1a = tf.subtract(my_one , h) 
    weight1b = tf.subtract(h , my_one)
    #weight2 = tf.subtract()
    #r = tf.matmul(tf.subtract(my_one , h) , z)
    elementwise_mul1a = tf.multiply(weight1a , z1a)
    elementwise_mul1b = tf.multiply(weight1b , z1b)


    return elementwise_mul1a + elementwise_mul1b

In [37]:
w1a = tf.Variable(tf.random_normal(shape=[Nfactor , 1] , stddev=0.1))

In [38]:
w1b = tf.Variable(tf.random_normal(shape=[Nfactor , 1] , stddev=0.1))

In [39]:
w2 = tf.Variable(tf.random_normal(shape=[Nregime_factor , 1] , stddev=0.1))

In [45]:
y_model = model(X , X2 , w1a , w1b , w2)

以下のふたつは同等？：

In [76]:
cost = tf.reduce_mean(tf.square(Y - y_model))

In [63]:
cost = tf.losses.mean_squared_error(labels=Y , predictions=y_model)

check用：

In [ ]:
cost = tf.add(tf.constant(dtype="float" , value = 100 ) , tf.reduce_mean(tf.square(Y - y_model)))

In [89]:
train_op = tf.train.GradientDescentOptimizer(0.1).minimize(cost) # construct an optimizer to minimize cost and fit line to my data

## run

In [90]:
sess = tf.Session()

sess.run(tf.global_variables_initializer())

In [115]:
sess.run(train_op , feed_dict={X:trX , X2:trX2 , Y : trY})

In [116]:
run_cost = sess.run(cost , feed_dict={X:trX , X2:trX2 , Y : trY})

In [117]:
run_cost

26.844284

In [65]:
y = sess.run(py , feed_dict={X:trX , X2:trX2})

In [56]:
trY.shape

AttributeError: 'list' object has no attribute 'shape'

In [43]:
y.shape

(1000, 1)

In [62]:
run_cost.shape

(1000, 1000)